In [31]:
#load libraries
import ee
import datetime
import fiona
from shapely.geometry import shape
import itertools
import os.path

In [42]:
#set important variables that probably do change from time to time
extent_shapefile_path = '/home/dan/Documents/react_data/Cities_React/Boundaries.shp'
modis_product = 'MODIS/006/MOD09A1'
product = 'MOD09A1'
start_year = 2004
end_year = 2016
end_year = end_year + 1 #because how range works
variables = ['sur_refl_b01','sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04','sur_refl_b05','sur_refl_b06','sur_refl_b07','QA','StateQA']
target_crs = 'SR-ORG:6974'

In [43]:
#load the shapefile
boundaries = fiona.open(extent_shapefile_path)
the_cities = list(boundaries.items())

In [44]:
#get a list of the city names
citynames = [item[1]['properties']['Name'] for item in the_cities]

In [45]:
#create a grid/dictionary of things to download
listolists = [variables, citynames, range(start_year,end_year)]
combos = list(itertools.product(*listolists))

In [47]:
#find which files are already downloaded
dlpath = '/media/dan/googledrive/gdrive/modis/'
filenames = [dlpath + c[1] + "_" + product +'_'+c[0]+'_'+str(c[2]) +'.tif'  for c in combos]
already_downloaded = [os.path.isfile(fff) for fff in filenames]

In [51]:
bla[1:5]
filenames[1:5]

[u'/media/dan/googledrive/gdrive/modis/Kinshasa_MOD11A2_LST_Day_1km_2005.tif',
 u'/media/dan/googledrive/gdrive/modis/Kinshasa_MOD11A2_LST_Day_1km_2006.tif',
 u'/media/dan/googledrive/gdrive/modis/Kinshasa_MOD11A2_LST_Day_1km_2007.tif',
 u'/media/dan/googledrive/gdrive/modis/Kinshasa_MOD11A2_LST_Day_1km_2008.tif']

In [4]:
#Start earth engine
ee.Initialize()
#pull the product
imgs = ee.ImageCollection(modis_product)

In [5]:
#for each product
for year in range(start_year, end_year):
    for vari in variables:
        start = str(year)+'-'+'01-01'
        end = str(year)+'-'+'12-31'
        dat = imgs.filterDate(start, end).select(vari).toList(1000);

        #for each city
        for cit in range(0,len(the_cities)):
            city_name = the_cities[cit][1]['properties']['Name']
            bbox = shape(the_cities[cit][1]['geometry']).bounds
            #print vari, city_name, bbox

            city_name = city_name.replace(' ','')
            
            #name the job
            job_id = city_name + "_" + product +'_'+vari+'_'+str(year)
            
            result = ee.Image.cat(dat.get(0))
            
            #create a polygon to clip things too
            # xMin, yMin, xMax, yMax
            target_projection = ee.Projection(target_crs)
            thecity = ee.Geometry.Rectangle(bbox) #.transform(target_projection,250)

            #convert an image collection into an image
            #also, save the dates for layer names later
            
            
            namefile = open('/media/dan/googledrive/' + job_id + '.txt', "w")
            namefile.write("%s\n" % result.get('system:index').getInfo())
            for i in range(1,dat.length().getInfo()):
                newimage = ee.Image.cat(dat.get(i))
                namefile.write("%s\n" % newimage.get('system:index').getInfo())
                result = ee.Image.cat([result, newimage])
            namefile.close()
                

            #result = result.clip(thecity)
            #request to download
            #https://stackoverflow.com/questions/39219705/how-to-download-images-using-google-earth-engines-python-api
            

            task = ee.batch.Export.image.toDrive(image = result, folder = 'modis', 
                                                 description = job_id, fileNamePrefix = job_id, region = thecity.getInfo().get('coordinates'))
            #print(job_id)
            task.start()   

EEException: Earth Engine memory capacity exceeded.

In [ ]:
#task.start()

In [ ]:
names[0]

In [ ]:
start

In [ ]:
end

In [ ]:
job_id

In [ ]:
namefile = open('/media/dan/googledrive/' + job_id + '.txt', "w")
for item in names:
  namefile.write("%s\n" % item)
namefile.close()